In [17]:
import sys
import math

# INSTRUCTIONS !
# Modify the traceit function to work for function calls instead of lines. 
# It should save the function name and the return value of the function 
# for each function call. 
# Use the mystery function "mystery(magic)" defined below and the
# given test cases for this exercise.
# Modify the provided functions to use this information to calculate
# phi values for the function calls as described in the video.
# You should get 3 phi values for each function - one for positive values (1),
# one for 0 values and one for negative values (-1), called "bins" in the video.
# When you have found out which function call and which return value type (bin)
# correlates the most with failure, fill in manually the following 3 variables,
# Do NOT set these values dynamically.


answer_function = "f3"   # One of f1, f2, f3
answer_bin = -1         # One of 1, 0, -1
answer_value = 0.8165   # precision to 4 decimal places.


# global variable to record the returned values of each function
returned = {}


def traceit(frame, event, arg):
    """Tracing function that tracks the returned values of function calls.
    
    The returned values are categorized in three categories as follows:
    -1: If the returned value is less than zero (for numbers), None, NaN or an Exception
     0: If the returned value is zero (for numbers), False or len(returned_value)==0 (for objects
        that have a __len__ method)
     1: If the returned value is greater than zero (for numbers), True or 
        len(returned_value)>0 (for objects that have a __len__ method)
        
    If the returned value is not included in the above, a TypeError is raised."""
    
    global returned
    
    if event == "return":
        function_name = frame.f_code.co_name
        return_value = arg
        
        # This check is only necessary when running this code in Jupyter notebook
        if function_name not in ["mystery", "f1", "f2", "f3", "test"]:
            return traceit
        
        if not returned.has_key(function_name):
            returned[function_name] = {-1: False, 0: False, 1: False}
        
        if isinstance(return_value, (int, long)):
            if return_value < 0:
                returned[function_name][-1] = True
            elif return_value == 0:
                returned[function_name][0] = True
            else:
                returned[function_name][1] = True
       
        elif isinstance(return_value, float):
            if math.isnan(return_value):
                returned[function_name][-1] = True
            elif return_value < 0:
                returned[function_name][-1] = True
            elif return_value == 0:
                returned[function_name][0] = True
            else:
                returned[function_name][1] = True
                
        elif hasattr(return_value, "__len__"):
            if len(return_value) == 0:
                returned[function_name][0] = True
            else:
                returned[function_name][1] = True
                
        elif return_value is True:
            returned[function_name][1] = True
            
        elif return_value is False:
            returned[function_name][0] = True
            
        elif return_value is None:
            returned[function_name][-1] = True
            
        elif isinstance(return_value, BaseException):
            returned[function_name][-1] = True
            
        else:
            raise TypeError, "Function {} returned an unknown object type".format(function_name)
        
    return traceit


def phi(n11, n10, n01, n00):
    """Calculate phi coefficient from given values."""
    return ((n11 * n00 - n10 * n01) / 
             math.sqrt((n10 + n11) * (n01 + n00) * (n10 + n00) * (n01 + n11)))


def print_tables(tables):
    """Print out values of phi, and result of runs for each covered function outcome."""
    
    for function_name in tables.keys():
        for i in [-1, 0, 1]:
            (n11, n10, n01, n00) = tables[function_name][i]
            try:
                factor = phi(n11, n10, n01, n00)
                prefix = "%+.4f%2d%2d%2d%2d" % (factor, n11, n10, n01, n00)
            except:
                prefix = "       %2d%2d%2d%2d" % (n11, n10, n01, n00)           
                    
            print prefix, "function name = {}".format(function_name), "result = {}".format(i)
                            

def run_tests(test_cases):
    """Run the 'mystery' function for each test case.
    Record and return input, outcome and function outcomes."""
    
    global returned
    runs = []
    
    for case in test_cases:
        returned = {}
        sys.settrace(traceit)
        outcome = mystery(case)
        sys.settrace(None)
        runs.append((case, outcome, returned))
    
    return runs


def init_tables(runs):
    """Create empty tuples for each function outcome."""
    
    tables = {}
    
    for (case, outcome, returned) in runs:
        for function_name, results in returned.items():
            if not tables.has_key(function_name):
                    tables[function_name] = {}
            for result, value in results.items():
                tables[function_name][result] = (0, 0, 0, 0)

    return tables


def compute_n(runs, tables):
    """Compute n11, n10, etc. for each function outcome."""
    
    for function_name, results in tables.items():
        
        for result in results.keys():
            (n11, n10, n01, n00) = tables[function_name][result]
            
            for (case, outcome, returned) in runs:       
                # Covered in this run
                if returned.has_key(function_name) and returned[function_name][result]:
                    if outcome == "FAIL":
                        n11 += 1  # covered and fails
                    else:
                        n10 += 1  # covered and passes
                # Not covered in this run
                else:
                    if outcome == "FAIL":
                        n01 += 1  # uncovered and fails
                    else:
                        n00 += 1  # uncovered and passes
            
            tables[function_name][result] = (n11, n10, n01, n00)
    
    return tables
      

def mystery(magic):
    assert type(magic) == tuple
    assert len(magic) == 3
    
    l, s, n = magic
    
    r1 = f1(l)
    
    r2 = f2(s)
    
    r3 = f3(n)

    if -1 in [r1, r2, r3]:
        return "FAIL"
    elif r3 < 0:
        return "FAIL"
    elif not r1 or not r2:
        return "FAIL"
    else:
        return "PASS"

def f1(ml):
    if type(ml) is not list:
        return -1
    elif len(ml) <6 :
        return len(ml)
    else:
        return 0
    
def f2(ms):    
    if type(ms) is not str:
        return -1
    elif len(ms) <6 :
        return len(ms)
    else:
        return 0

def f3(mn):
    if type(mn) is not int:
        return -1
    if mn > 10:
        return -100
    else:
        return mn


# These are the input values you should test the mystery function with
inputs = [([1,2],"ab", 10), 
          ([1,2],"ab", 2),
          ([1,2],"ab", 12),
          ([1,2],"ab", 21),
          ("a",1, [1]),
          ([1],"a", 1),
          ([1,2],"abcd", 8),
          ([1,2,3,4,5],"abcde", 8),
          ([1,2,3,4,5],"abcdefgijkl", 18),
          ([1,2,3,4,5,6,7],"abcdefghij", 5)]



runs = run_tests(inputs)

tables = init_tables(runs)
tables = compute_n(runs, tables)

print_tables(tables)      

        0 0 5 5 function name = mystery result = -1
        0 0 5 5 function name = mystery result = 0
        5 5 0 0 function name = mystery result = 1
+0.3333 1 0 4 5 function name = f1 result = -1
+0.3333 1 0 4 5 function name = f1 result = 0
-0.5000 3 5 2 0 function name = f1 result = 1
+0.3333 1 0 4 5 function name = f2 result = -1
+0.5000 2 0 3 5 function name = f2 result = 0
-0.6547 2 5 3 0 function name = f2 result = 1
+0.8165 4 0 1 5 function name = f3 result = -1
        0 0 5 5 function name = f3 result = 0
-0.8165 1 5 4 0 function name = f3 result = 1
